In [2]:
from ultralytics import YOLO
import cv2
import csv
from pathlib import Path
import time

MODEL_PATH = 'yolov8n.pt'
VIDEO_PATH = '/home/linux/coding/elc_activities/computer_vision/mixkit-ride-through-the-streets-of-london-4263-hd-ready.mp4'
CONF_THRESHOLD = 0.3
RESIZE_SCALE = 0.75
TRACKER_CFG = 'bytetrack.yaml'
CSV_OUTPUT = 'vehicle_counts.csv'

model = YOLO(MODEL_PATH)
CLASS_NAMES = model.model.names or {i: f'class_{i}' for i in range(model.model.model.nc)}
normalize_class = lambda name: name.lower().replace(" ", "").strip()

LMV_CLASSES = {'car', 'motorbike', 'motorcycle', 'auto', 'autorickshaw', 'scooter'}
HMV_CLASSES = {'bus', 'truck', 'tractor'}
LMV_IDS = {k for k, v in CLASS_NAMES.items() if normalize_class(v) in LMV_CLASSES}
HMV_IDS = {k for k, v in CLASS_NAMES.items() if normalize_class(v) in HMV_CLASSES}

cap = cv2.VideoCapture(VIDEO_PATH)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) * RESIZE_SCALE)
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) * RESIZE_SCALE)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

video_name = Path(VIDEO_PATH).stem
out = cv2.VideoWriter(f'{video_name}_out.avi', cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))

Path(CSV_OUTPUT).unlink(missing_ok=True)
csv_file = open(CSV_OUTPUT, 'w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Frame', 'LMV Count', 'HMV Count'])

def count_vehicles(boxes):
    lmv, hmv = 0, 0
    for box in boxes:
        cls_id = int(box.cls[0])
        if cls_id in LMV_IDS:
            lmv += 1
        elif cls_id in HMV_IDS:
            hmv += 1
    return lmv, hmv

def draw_boxes(frame, boxes):
    for box in boxes:
        conf = box.conf[0].item()
        if conf < CONF_THRESHOLD:
            continue
        cls_id = int(box.cls[0])
        if cls_id not in LMV_IDS | HMV_IDS:
            continue
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        track_id = int(box.id[0]) if box.id is not None else None
        vehicle_type = 'LMV' if cls_id in LMV_IDS else 'HMV'
        color = (0, 255, 0) if vehicle_type == 'LMV' else (0, 0, 255)
        label = f"{CLASS_NAMES[cls_id]} | ID:{track_id} | {vehicle_type} | {conf:.2f}" if track_id else f"{CLASS_NAMES[cls_id]} | {vehicle_type} | {conf:.2f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, max(y1 - 10, 20)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

def process_frame(frame):
    resized = cv2.resize(frame, (frame_width, frame_height))
    result = model.track(
        resized,
        tracker=TRACKER_CFG,
        persist=True,
        verbose=False,
        conf=CONF_THRESHOLD,
        iou=0.5
    )[0]
    if result and result.boxes is not None:
        boxes = result.boxes
        draw_boxes(resized, boxes)
        lmv, hmv = count_vehicles(boxes)
    else:
        lmv, hmv = 0, 0
    return resized, lmv, hmv

frame_idx = 0
start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    processed_frame, lmv_count, hmv_count = process_frame(frame)
    cv2.putText(processed_frame, f"LMV: {lmv_count}", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(processed_frame, f"HMV: {hmv_count}", (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.putText(processed_frame, f"Total: {lmv_count + hmv_count}", (20, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    csv_writer.writerow([frame_idx, lmv_count, hmv_count])
    out.write(processed_frame)
    elapsed_time = time.time() - start_time
    fps_display = frame_idx / elapsed_time if elapsed_time > 0 else 0
    cv2.putText(processed_frame, f"FPS: {fps_display:.2f}", (frame_width - 150, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
    cv2.imshow("YOLOv8 + ByteTrack Vehicle Counting", processed_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    frame_idx += 1

cap.release()
out.release()
csv_file.close()
cv2.destroyAllWindows()
